<a href="https://colab.research.google.com/github/ujjawalsingh10/Neural-Machine-Translation/blob/main/Neural_Machine_Translation_with_Bahdanau_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.layers import InputLayer, MaxPool2D, Dense, Conv2D, Conv1D, Flatten, BatchNormalization, TextVectorization,SimpleRNN, Embedding, Input,Bidirectional, LSTM, Dropout, GRU
from google.colab import drive
import re
import string
from numpy import random
import gensim.downloader as api
import datetime
from tensorboard.plugins import projector
import os
import pandas as pd
from tensorflow.keras import Model

### Data Preparation

In [3]:
!wget http://www.manythings.org/anki/fra-eng.zip

--2023-07-18 16:50:35--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7420323 (7.1M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.08M  32.5MB/s    in 0.2s    

2023-07-18 16:50:36 (32.5 MB/s) - ‘fra-eng.zip’ saved [7420323/7420323]



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# !unzip '/content/fra-eng.zip' -d '/content/drive/MyDrive/Deep_Learning/NLP/Neural_Machine_Translation_with_RNN/dataset'

### Data Processing

In [6]:
### To convert our dataset into TensorFlow dataset types for easy manipulation
text_dataset = tf.data.TextLineDataset('/content/drive/MyDrive/Deep_Learning/NLP/Neural_Machine_Translation_with_RNN/dataset/fra.txt')

In [7]:
for i in text_dataset.take(3):
  print(i)

tf.Tensor(b'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)', shape=(), dtype=string)
tf.Tensor(b'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)', shape=(), dtype=string)
tf.Tensor(b'Go.\tEn route !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)', shape=(), dtype=string)


In [8]:
VOCAB_SIZE = 20000
ENGLISH_SEQUENCE_LENGTH = 64
FRENCH_SEQUENCE_LENGTH = 64
EMBEDDING_DIM = 300
BATCH_SIZE = 64

In [9]:
# ### We can check last of the elements to see what can be the max size of the sentences
# for i in text_dataset.skip(190000):
#   print(len(tf.strings.split(i, ' ')))

In [10]:
english_vectorize_layer = TextVectorization(
    standardize = 'lower_and_strip_punctuation',
    max_tokens = VOCAB_SIZE,
    output_mode = 'int',
    output_sequence_length = ENGLISH_SEQUENCE_LENGTH
)

In [11]:
french_vectorize_layer = TextVectorization(
    standardize = 'lower_and_strip_punctuation',
    max_tokens = VOCAB_SIZE,
    output_mode = 'int',
    output_sequence_length = FRENCH_SEQUENCE_LENGTH
)

In [12]:
for i in text_dataset.take(1):
  print(i)

tf.Tensor(b'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)', shape=(), dtype=string)


We have to create one vocabulary for English and another for Hindi

In [14]:
### We create this method to get data in x,y format and get rid of the extras z
## We add tokens and change the dataset to 3 input type
def selector(input_text):
  split_text = tf.strings.split(input_text, '\t')
  return {'input_1' : split_text[0:1], 'input_2' : 'starttoken '+split_text[1:2] }, split_text[1:2]+' endtoken'

In [15]:
split_dataset = text_dataset.map(selector)

In [16]:
def separator(input_text):
  split_text = tf.strings.split(input_text, '\t')
  return split_text[0:1], 'starttoken '+ split_text[1:2]+ ' endtoken'

In [20]:
init_dataset = text_dataset.map(separator)

In [21]:
for i in split_dataset.take(3):
  print(i)

({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken Va !'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Va ! endtoken'], dtype=object)>)
({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken Marche.'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Marche. endtoken'], dtype=object)>)
({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, 'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken En route !'], dtype=object)>}, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'En route ! endtoken'], dtype=object)>)


In [22]:
english_training_dataset = init_dataset.map(lambda x,y : x)
english_vectorize_layer.adapt(english_training_dataset)

In [23]:
french_training_dataset = init_dataset.map(lambda x,y : y)
french_vectorize_layer.adapt(french_training_dataset)

In [24]:
# def vectorizer(english, french):
#   return english_vectorize_layer(english), french_vectorize_layer(french)
def vectorizer(inputs, output):
  return {'input_1': english_vectorize_layer(inputs['input_1']),
          'input_2': french_vectorize_layer(inputs['input_2'])}, french_vectorize_layer(output)

In [25]:
dataset = split_dataset.map(vectorizer)

In [26]:
for i in dataset.take(3):
  print(i)

({'input_1': <tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[44,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])>, 'input_2': <tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[  2, 104,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>}, <tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[104,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,

In [27]:
french_vectorize_layer.get_vocabulary()[104], english_vectorize_layer.get_vocabulary()[44]

('va', 'go')

In [28]:
french_vectorize_layer.get_vocabulary()[104], english_vectorize_layer.get_vocabulary()[44]

('va', 'go')

In [29]:
dataset = dataset.shuffle(2048).unbatch().batch(BATCH_SIZE).prefetch(buffer_size = tf.data.AUTOTUNE)

In [30]:
NUM_BATCHES = int(200000/BATCH_SIZE)

In [31]:
train_dataset = dataset.take(int(0.9 * NUM_BATCHES))
val_dataset = dataset.skip(int(0.9 * NUM_BATCHES))

### Model Building

In [32]:
NUM_UNITS = 256